In [2]:
import os, os.path
from billboard_scrapping import get_titles_and_artists_billboard, clean_artist, clean_song
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen, ProxyHandler, build_opener
import time
from whoosh.fields import Schema, TEXT, ID, STORED
from whoosh.query import Every
from whoosh.index import create_in, open_dir, exists_in
from whoosh.qparser import QueryParser
from shutil import rmtree
import re
from whoosh.query import Phrase

In [3]:
base_url = 'https://www.azlyrics.com/'
headers = {'User-Agent':'Mozilla/5.0'}

def get_song_data_from_url(url):
    try:
        req = Request(url, headers=headers)
        #req.set_proxy(proxy_host, 'http')
        webpage = urlopen(req).read()
        bs_webpage = BeautifulSoup(webpage,"lxml")
    except:
        print('exception at url: {}'.format(url))
        return None
        
    title = bs_webpage.find_all('b')[1].text
    artist = bs_webpage.find('h2').find('b').text
    lyrics = bs_webpage.find_all('div', attrs={'class': None})[1].text
    
    try:
        album = bs_webpage.find('div', class_='songinalbum_title').text
    except AttributeError:
        album = ""
    
    cleaning = Cleaning()
    title = cleaning.title(title)
    lyrics = cleaning.lyrics(lyrics)
    full_lyrics = cleaning.full_lyrics(lyrics)
    
    if album=="You May Also Like":
        album = ""
    if album!="":
        album = cleaning.album(album)
    
    data = {
        'title': title,
        'artist': artist,
        'lyrics': lyrics,
        'full_lyrics': full_lyrics,
        'album': album,
        'url': url
    }
    
    return data

def create_schema():
    schema = Schema(url=ID(),
                    title=TEXT(stored=True),
                    artist=TEXT(stored=True),
                    full_lyrics=TEXT(stored=True, phrase=True),
                    lyrics=TEXT(stored=True),
                    album=TEXT(stored=True))
    return schema

def create_or_open_index(directory):
    if not os.path.exists(directory):
        os.mkdir(directory)
    if exists_in(directory):
        index = open_dir(directory)
    else:
        schema = create_schema()
        index = create_in(directory, schema)
    return index

def index_song(index, song_data):
    writer = index.writer()
    writer.add_document(url=u'{}'.format(song_data['url']),
                        title=u'{}'.format(song_data['title']),
                        artist=u'{}'.format(song_data['artist']),
                        full_lyrics=u'{}'.format(song_data['full_lyrics']),
                        lyrics=u'{}'.format(song_data['lyrics']),
                        album=u'{}'.format(song_data['album']))
    writer.commit(optimize=True)
    
def search_song_by_title(title, index):
    results_list = list()
    qp = QueryParser('title', schema=index.schema)
    q = qp.parse(u"{}".format(title))
    with index.searcher() as searcher:
        results = searcher.search(q)
        for result in results:
            data = {
                'title': result['title'],
                'artist': result['artist'],
                'full_lyrics': result['full_lyrics'],
                'lyrics': result['lyrics'],
                'album': result['album']
            }
            results_list.append(data)
    return results_list

def search_song_by_author(author, index):
    results_list = list()
    qp = QueryParser('author', schema=index.schema)
    q = qp.parse(u"{}".format(title))
    with index.searcher() as searcher:
        results = searcher.search(q)
        for result in results:
            data = {
                'title': result['title'],
                'artist': result['artist'],
                'full_lyrics': result['full_lyrics'],
                'lyrics': result['lyrics'],
                'album': result['album']
            }
            results_list.append(data)
    return results_list

def search_song_by_lyrics(terms, index):
    results_list = list()
    qp = QueryParser('full_lyrics', schema=index.schema)
    q = qp.parse(u'"{}"'.format(terms))
    
    with index.searcher() as searcher:
        results = searcher.search(q)
        for result in results:
            data = {
                'title': result['title'],
                'artist': result['artist'],
                'full_lyrics': result['full_lyrics'],
                'lyrics': result['lyrics'],
                'album': result['album']
            }
            results_list.append(data)
    return results_list

def get_songs_urls_by_letter(letter, limit=None):

    url = base_url+'{}.html'.format(letter)
    final_url_list = list()
    
    def get_urls_by_letter(url):
        req = Request(url)
        webpage = urlopen(req).read()
        bs_webpage = BeautifulSoup(webpage,"lxml")
        divs = bs_webpage.find_all('div', class_='col-sm-6')
        a_list = [i.find_all('a') for i in divs]
        a_list = [base_url+i['href'] for j in a_list for i in j]
        return a_list
    
    urls = get_urls_by_letter(url)
    
    for u in urls[:limit]:
        req = Request(u)
        webpage = urlopen(req).read()
        bs_webpage = BeautifulSoup(webpage,"lxml")
        divs = bs_webpage.find_all('div', class_='listalbum-item')
        a_list = [i.find_all('a') for i in divs]
        final_url_list.append([base_url+i['href'][3:] for j in a_list for i in j])
        time.sleep(20)
    
    final_url_list = [i for j in final_url_list for i in j]
    
    return final_url_list

def index_songs_by_letter(letter, index, limit=None):
    urls = get_songs_urls_by_letter(letter, limit)
    for u in urls:
        song_data = get_song_data_from_url(u)
        print(song_data['title'])
        index_song(index, song_data)
        time.sleep(15)
        
def index_songs_by_artist(artist, index):
    letter = 19 if artist[:1] not in "abcdefghijklmnopqrstuvwxyz" else artist[:1]
    url = base_url+"{}/{}.html".format(letter, clean_artist(artist))
    
    req = Request(url)
    webpage = urlopen(req).read()
    bs_webpage = BeautifulSoup(webpage,"lxml")
    divs = bs_webpage.find_all('div', class_='listalbum-item')
    a_list = [i.find_all('a') for i in divs]
    final_url_list = [base_url+i['href'][3:] for j in a_list for i in j]
    time.sleep(20)
    
    for u in final_url_list:
        song_data = get_song_data_from_url(u)
        print(song_data['title'])
        index_song(index, song_data)
        time.sleep(15)
        
def index_songs_by_billboard(number, index, limit=None):
    song_artist_tuple = get_titles_and_artists_billboard(number)
    
    for song, artist in song_artist_tuple:
        song = clean_song(song)
        artist = clean_artist(artist)
        url = base_url+'lyrics/{}/{}.html'.format(artist, song)
        print(url)
        song_data = get_song_data_from_url(url)
        if song_data is None:
            continue
        print(song_data['title'])
        index_song(index, song_data)
        time.sleep(15)

class Cleaning():
    
    def full_lyrics(self, lyrics):
        lyrics = lyrics.split('\r\n')
        lyrics = [i.replace('\n', ' ')
                  for i in lyrics if i not in ['\n', '\r', '\n\r', '\r\n', '']]
        
        lyrics = ', '.join(lyrics).replace(',', '').replace('.', '').lower()
    
        return lyrics
    
    def lyrics(self, lyrics):
        lyrics = lyrics.replace('\r', '').replace('\n\n', '\n')
        lyrics = lyrics[1:][:-1]
        return lyrics
    
    def title(self, title):
        title = title.replace('"', '')
        return title
    
    def album(self, album):
        print(album)
        if (album!=""):
            album = re.findall(r'"([^"]*)"', album)[0]
        return album

In [4]:
# Ejemplo para una url de una cancion

url="https://www.azlyrics.com/lyrics/pablocruise/islandwoman.html"

song_data = get_song_data_from_url(url)
rmtree('./index')
index = create_or_open_index('./index')
index_song(index, song_data)
results = search_song_by_title('Island', index)
#results

In [ ]:
# Ejemplo para urls por letra

rmtree('./index')
index = create_or_open_index('./index')
for letter in "abcdefghijklmnopqrstuvwxyz#":
    print(letter)
    index_songs_by_letter(letter, index)

In [ ]:
# Ejemplo para urls por billboard

rmtree('./index')
index = create_or_open_index('./index')
index_songs_by_billboard(100, index)

In [8]:
# Ejemplo para urls por billboard

# rmtree('./index')
index = create_or_open_index('./index')
index_songs_by_artist('nathy peluso', index)

mixtape: "Esmeralda" (2017)
Alabame
mixtape: "Esmeralda" (2017)
Dafne
mixtape: "Esmeralda" (2017)
Daga
mixtape: "Esmeralda" (2017)
Esmeralda
mixtape: "Esmeralda" (2017)
Oreen Ishi
mixtape: "Esmeralda" (2017)
Sandía
mixtape: "Esmeralda" (2017)
Kun Fu
EP: "La Sandunguera" (2018)
Estoy Triste
EP: "La Sandunguera" (2018)
Hot Butter
EP: "La Sandunguera" (2018)
Gimme Some Pizza
EP: "La Sandunguera" (2018)
La Sandunguera
EP: "La Sandunguera" (2018)
Ma Time
EP: "La Sandunguera" (2018)
La Passione
album: "Calambre" (2020)
Celebré
album: "Calambre" (2020)
Sana Sana
album: "Calambre" (2020)
Buenos Aires
album: "Calambre" (2020)
Delito
album: "Calambre" (2020)
Sugga
album: "Calambre" (2020)
Trío
album: "Calambre" (2020)
Business Woman
album: "Calambre" (2020)
Llámame
album: "Calambre" (2020)
Amor Salvaje
album: "Calambre" (2020)
Arrorró
album: "Calambre" (2020)
Puro Veneno
album: "Calambre" (2020)
Agárrate
Copa Glasé
Corashe
exception at url: https://www.azlyrics.com/ps://www.azlyrics.com/lyrics/b

TypeError: 'NoneType' object is not subscriptable